In [2]:
!python2 code/preprocess.py dataset/train-processed.csv #Process train data

Processing 100000/100000
Saved processed tweets to: dataset/train-processed-processed.csv


In [3]:
!python2 code/preprocess.py dataset/test-processed.csv #Process test data

Traceback (most recent call last):
  File "code/preprocess.py", line 107, in <module>
    preprocess_csv(csv_file_name, processed_file_name, test_file=False)
  File "code/preprocess.py", line 81, in preprocess_csv
    positive = int(line[:line.find(',')])
ValueError: invalid literal for int() with base 10: 'is so sad for my apl friend'


In [4]:
!python2 code/stats.py dataset/train-processed-processed.csv #Data train

Processing 100000/100000
Calculating frequency distribution
Saved uni-frequency distribution to dataset/train-processed-processed-freqdist.pkl
Saved bi-frequency distribution to dataset/train-processed-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 100000, Positive: 56462, Negative: 43538
User Mentions => Total: 0, Avg: 0.0000, Max: 0
URLs => Total: 0, Avg: 0.0000, Max: 0
Emojis => Total: 0, Positive: 0, Negative: 0, Avg: 0.0000, Max: 0
Words => Total: 1283269, Unique: 50361, Avg: 12.8327, Max: 41, Min: 0
Bigrams => Total: 1183437, Unique: 392543, Avg: 11.8344


In [5]:
!python2 code/stats.py dataset/test-processed-processed.csv #New data test


Calculating frequency distribution
Saved uni-frequency distribution to dataset/test-processed-processed-freqdist.pkl
Saved bi-frequency distribution to dataset/test-processed-processed-freqdist-bi.pkl

[Analysis Statistics]
Tweets => Total: 0, Positive: 0, Negative: 0
Traceback (most recent call last):
  File "code/stats.py", line 106, in <module>
    print 'User Mentions => Total: %d, Avg: %.4f, Max: %d' % (num_mentions, num_mentions / float(num_tweets), max_mentions)
ZeroDivisionError: float division by zero


In [8]:
!python2 code/baseline.py TRAIN = True #use baseline

Correct = 65.32%


In [12]:
!python2 code/naivebayes.py TRAIN = True #Classification

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7794/10000 = 77.9400 %


In [29]:
!python3 code/logistic.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
2022-01-24 09:52:25.275912: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Iteration 105/180, loss:0.6246, acc:0.71202022-01-24 09:52:40.114219: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 180/180, loss:0.6080, acc:0.7100
Epoch: 1, val_acc:0.7318
Accuracy improved from 0.0000 to 0.7318, saving model
Iteration 10/180, loss:0.5838, acc:0.75602022-01-24 09:52:52.708934: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 94/180, loss:0.5669, acc:0.74002022-01-24 09:53:02.738087: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 99/180, loss:0.5818, acc:0.73602022

In [30]:
!python2 code/decisiontree.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 6792/10000 = 67.9200 %


In [31]:
!python2 code/randomforest.py TRAIN = True #Classification using random forest

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Testing
Processing 1/1
Correct: 7195/10000 = 71.9500 %


Using XGBoost

In [32]:
import utils
import random
import numpy as np
from xgboost import XGBClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

# Performs classification using XGBoost.


FREQ_DIST_FILE = 'dataset/train-processed-processed-freqdist.pkl'
BI_FREQ_DIST_FILE = 'dataset/train-processed-processed-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = 'dataset/train-processed-processed.csv'
TEST_PROCESSED_FILE = 'dataset/test-processed-processed.csv'
TRAIN = True
UNIGRAM_SIZE = 1500
VOCAB_SIZE = UNIGRAM_SIZE
USE_BIGRAMS = True
if USE_BIGRAMS:
    BIGRAM_SIZE = 100
    VOCAB_SIZE = UNIGRAM_SIZE + BIGRAM_SIZE
FEAT_TYPE = 'frequency'


def get_feature_vector(tweet):
    uni_feature_vector = []
    bi_feature_vector = []
    words = tweet.split()
    for i in range(len(words) - 1):
        word = words[i]
        next_word = words[i + 1]
        if unigrams.get(word):
            uni_feature_vector.append(word)
        if USE_BIGRAMS:
            if bigrams.get((word, next_word)):
                bi_feature_vector.append((word, next_word))
    if len(words) >= 1:
        if unigrams.get(words[-1]):
            uni_feature_vector.append(words[-1])
    return uni_feature_vector, bi_feature_vector


def extract_features(tweets, batch_size=500, test_file=True, feat_type='presence'):
    num_batches = int(np.ceil(len(tweets) / float(batch_size)))
    for i in range(num_batches):
        batch = tweets[i * batch_size: (i + 1) * batch_size]
        features = lil_matrix((batch_size, VOCAB_SIZE))
        labels = np.zeros(batch_size)
        for j, tweet in enumerate(batch):
            if test_file:
                tweet_words = tweet[1][0]
                tweet_bigrams = tweet[1][1]
            else:
                tweet_words = tweet[2][0]
                tweet_bigrams = tweet[2][1]
                labels[j] = tweet[1]
            if feat_type == 'presence':
                tweet_words = set(tweet_words)
                tweet_bigrams = set(tweet_bigrams)
            for word in tweet_words:
                idx = unigrams.get(word)
                if idx:
                    features[j, idx] += 1
            if USE_BIGRAMS:
                for bigram in tweet_bigrams:
                    idx = bigrams.get(bigram)
                    if idx:
                        features[j, UNIGRAM_SIZE + idx] += 1
        yield features, labels


def apply_tf_idf(X):
    transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
    transformer.fit(X)
    return transformer


def process_tweets(csv_file, test_file=True):
    """Returns a list of tuples of type (tweet_id, feature_vector)
            or (tweet_id, sentiment, feature_vector)
    Args:
        csv_file (str): Name of processed csv file generated by preprocess.py
        test_file (bool, optional): If processing test file
    Returns:
        list: Of tuples
    """
    tweets = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append((tweet_id, feature_vector))
            else:
                tweets.append((tweet_id, int(sentiment), feature_vector))
            utils.write_status(i + 1, total)
    print ('\n')
    return tweets


if __name__ == '__main__':
    np.random.seed(20)
    unigrams = utils.top_n_words(FREQ_DIST_FILE, UNIGRAM_SIZE)
    if USE_BIGRAMS:
        bigrams = utils.top_n_bigrams(BI_FREQ_DIST_FILE, BIGRAM_SIZE)
    tweets = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    if TRAIN:
        train_tweets, val_tweets = utils.split_data(tweets)
    else:
        random.shuffle(tweets)
        train_tweets = tweets
    del tweets
    print ('Extracting features & training batches')
    clf = XGBClassifier(max_depth=25, silent=False, n_estimators=20)
    batch_size = len(train_tweets)
    i = 1
    n_train_batches = int(np.ceil(len(train_tweets) / float(batch_size)))
    for training_set_X, training_set_y in extract_features(train_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
        utils.write_status(i, n_train_batches)
        i += 1
        if FEAT_TYPE == 'frequency':
            tfidf = apply_tf_idf(training_set_X)
            training_set_X = tfidf.transform(training_set_X)
        clf.fit(training_set_X, training_set_y)
    print ('\n')
    print ('Testing')
    if TRAIN:
        correct, total = 0, len(val_tweets)
        i = 1
        batch_size = len(val_tweets)
        n_val_batches = int(np.ceil(len(val_tweets) / float(batch_size)))
        for val_set_X, val_set_y in extract_features(val_tweets, test_file=False, feat_type=FEAT_TYPE, batch_size=batch_size):
            if FEAT_TYPE == 'frequency':
                val_set_X = tfidf.transform(val_set_X)
            prediction = clf.predict(val_set_X)
            correct += np.sum(prediction == val_set_y)
            utils.write_status(i, n_val_batches)
            i += 1
        print ('\nCorrect: %d/%d = %.4f %%' % (correct, total, correct * 100. / total))
    else:
        del train_tweets
        test_tweets = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        n_test_batches = int(np.ceil(len(test_tweets) / float(batch_size)))
        predictions = np.array([])
        print ('Predicting batches')
        i = 1
        for test_set_X, _ in extract_features(test_tweets, test_file=True, feat_type=FEAT_TYPE):
            if FEAT_TYPE == 'frequency':
                test_set_X = tfidf.transform(test_set_X)
            prediction = clf.predict(test_set_X)
            predictions = np.concatenate((predictions, prediction))
            utils.write_status(i, n_test_batches)
            i += 1
        predictions = [(str(j), int(predictions[j]))
                       for j in range(len(test_tweets))]
        utils.save_results_to_csv(predictions, 'xgboost.csv')
        print ('\nSaved to xgboost.csv')

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7275/10000 = 72.7500 %


In [33]:
!python2 code/svm.py TRAIN = True #model svm

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
Processing 1/1

Testing
Processing 1/1
Correct: 7848/10000 = 78.4800 %


In [34]:
!python3 code/neuralnet.py TRAIN = True

Generating feature vectors
Processing 100000/100000

Extracting features & training batches
2022-01-24 10:04:47.779436: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Iteration 66/180, loss:0.5761, acc:0.71602022-01-24 10:04:54.060183: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled
Iteration 180/180, loss:0.5063, acc:0.76402022-01-24 10:05:03.593761: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled

Epoch: 1, val_acc:0.7629
Accuracy improved from 0.0000 to 0.7629, saving model
Iteration 180/180, loss:0.4847, acc:0.7720
Epoch: 2, val_acc:0.7703
Accuracy improved from 0.7629 to 0.7703, saving model
Iteration 180/180, loss:0.4613, acc:0.7900
Epoch: 3, val_acc:0.7698
Iteration 180/180, loss:0.4912, acc:0.7760
Epoch: 4, val_acc:0.7698
Ite

In [35]:
!wget http://nlp.stanford.edu/data/glove.6B.zip #Download file

--2022-01-24 10:06:26--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-01-24 10:06:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-24 10:06:27--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [36]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [37]:
!python3 code/lstm.py

Looking for GLOVE vectors
Processing 400000/0

Found 31735 words in GLOVE
Generating feature vectors
Processing 100000/100000

2022-01-24 10:11:37.033724: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 200)           18000200  
                                                                 
 dropout (Dropout)           (None, 40, 200)           0         
                                                                 
 lstm (LSTM)                 (None, 128)               168448    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                         

In [38]:
!python3 code/cnn.py #CNN Model with 4 Convolution Layer Neural Network

Looking for GLOVE seeds
Processing 400000/0

Generating feature vectors
Processing 100000/100000

2022-01-24 10:15:15.717393: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch 1/8
704/704 [==============================] - 27s 26ms/step - loss: 0.5313 - accuracy: 0.7253 - val_loss: 0.4671 - val_accuracy: 0.7773
Epoch 2/8
704/704 [==============================] - 18s 25ms/step - loss: 0.4498 - accuracy: 0.7881 - val_loss: 0.4582 - val_accuracy: 0.7859
Epoch 3/8
704/704 [==============================] - 18s 25ms/step - loss: 0.3990 - accuracy: 0.8198 - val_loss: 0.4702 - val_accuracy: 0.7756
Epoch 4/8
704/704 [==============================] - 18s 26ms/step - loss: 0.3513 - accuracy: 0.8447 - val_loss: 0.4733 - val_accuracy: 0.7731
Epoch 5/8
704/704 [==============================] - 18s 25ms/step - loss: 0.3098 - accuracy: 0.8646 - val_loss: